<a href="https://colab.research.google.com/github/avinashmane/colab/blob/master/strava_likers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Code/Parameters
* Setup
    * needs
        * auth
        * gsheet auth
        * cfg_strava
* updates sheets
* Todo
    * low: error: location not found 

## Config

In [1]:
repo='https://github.com/avinashmane/colab.git'
import os,sys
if os.name=='nt':
    LIB= r'c:\m\lib'
    LOG= r'c:\m\social-media-automation'
    AUTH= r'c:\i\auth'
    sys.path.append("./lib")
else:
    if not os.path.exists('drive'):
        from google.colab import drive
        drive.mount('/content/drive')
    LIB='/content/drive/MyDrive/m/lib'
    LOG='/content/drive/MyDrive/m/lib'
    AUTH='/content/drive/MyDrive/m/auth'
    if not os.path.exists('m'):
      !ln -sd '/content/drive/MyDrive/m/' m
    if not os.path.exists('repo'):
      !git clone $repo repo
    sys.path.append('repo')
    sys.path.append("repo/lib")
sys.path.append(LIB)


In [2]:
import yaml
cfg=yaml.load(u"""
repo:
    https://github.com/avinashmane/colab.git
packages: 
- chromium-chromedriver
modules:
- nerodia
- pygsheets
env:
    nt:
        LIB: c:/m/lib
        LOG: c:/m/social-media-automation
        AUTH: c:/i/auth
    posix:
        LIB: /content/drive/MyDrive/m/lib
        LOG: /content/drive/MyDrive/m/lib
        AUTH: /content/drive/MyDrive/m/auth
sheets:
    stravasync:
        url: https://docs.google.com/spreadsheets/d/1Q8SA641P0aytWzPGmeHG_xuxll3Wq6phsxjb91RjgWY/edit#gid=0
        tab: Members
        range: A10:O
    StravaPromoLog:
        url: https://docs.google.com/spreadsheets/d/1Q8SA641P0aytWzPGmeHG_xuxll3Wq6phsxjb91RjgWY/edit#gid=0
        tab: StravaPromoLog
        
""",Loader=yaml.Loader)
for x in cfg['env'][os.name]: 
    os.environ[x]=eval(x)

import importlib
import utils
importlib.reload(utils)
utils.setup_env(cfg['packages'],cfg['modules'])

'Environment setup'

## Code

In [3]:
import yaml
from datetime import datetime
import os
from nerodia.browser import Browser
import nerodia
import re
import logging
from IPython.display import Image

import utils

import socialmediasite 
importlib.reload(socialmediasite)
Facebook=socialmediasite.Facebook

# Process
## Create headless

In [4]:
" Test Selenium headless"
if True:#'b' in globals():
    if os.name=='nt':
        b=Browser('chrome',)
    else:
        
        # import sys
        # sys.path.insert(0,f'{DIR}/geckodriver')
        from selenium import webdriver

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        # b =webdriver.Chrome('chromedriver',options=chrome_options)
        b=Browser('chrome',options=chrome_options,)
    
    utils.setLogger(f"{LOG}/strava-automation.log")
    

2022-08-20 23:57:09:INFO:Logger file: c:\m\social-media-automation/strava-automation.log


In [5]:
# %debug
# if os.name=='nt':
nerodia.default_timeout=3
fb=Facebook(b,authFile=f'{AUTH}/auth.yaml')


In [6]:
for _el in b.inputs(name='uid'):#(text='Log in to another account'):
    # _el.click()
    print(_el.attributes)

In [66]:
b.goto('m.facebook.com')
username="avinash"
b.cookies.load(f'{LIB}/facebook_{username}.cookies')
# b.cookies.clear()

In [58]:
def login():
    # user id selection
    for _el in b.inputs(name='uid'):
        logging.info("Login saved")
        _el.parent(tag_name='form').submit()
        b.input(name="pass").send_keys("Imfr0mk0p")
        b.button(value="Log In").click()
        break
    else:  # start from user id
        logging.info("Login not saved..entering email")
        b.input(id="m_login_email").send_keys("avinashmane@yahoo.com")

        b.input(id="m_login_password").send_keys("Imfr0mk0p")
        # b.button(name="login").send_keys(Keys.ENTER)
        #alternately 
        b.button(name="login").click()

2022-08-21 00:41:23:INFO:Login saved


In [59]:
b.goto('m.facebook.com')

'http://m.facebook.com'

In [60]:
username="avinash"
b.cookies.save(f'{LIB}/facebook_{username}.cookies')


In [61]:
os.environ["FB_2FA_CODE"]='210468'

In [62]:
"Loging"
tag=yaml.safe_load("""
user: class_name="_4kk6"
""")
tag

{'user': 'class_name="_4kk6"'}

In [68]:
import json
def getPostInfo(x):
    el={}
    data_ft=json.loads(x.data_ft)
    
    for _a in ['pos','is_sponsored','top_level_post_id','content_owner_id_new','page_id','photo_id']:
        if _a in data_ft:
            el[_a]=data_ft[_a]
    el['key']=x.data_dedupekey
    # el['hdrs']=len(x.headers())
    # el['ftrs']=len(x.footers())    
    for _el1 in x.divs(class_name="story_body_container"):
        # header first line
        for _el2 in _el1.headers():
            for _el3 in _el2.element(data_gt='{"tn":"C"}').links(class_name="_4kk6"):
                el['usr']=_el3.text[:60]
                el['usrUrl']=re.split(r"[\&\?]refid=",_el3.href)[0]
                # break
            # header second line
            #links(class_name="_26yo"):
            el['line2']=utils.getAttrsIfExists(_el2.abbr(),['text'])
            el['text'] =utils.getAttrsIfExists(_el2.following_sibling(),['text'])

    for _el2 in x.footers():
        # print(_el2)
        #footer status first line
        el['likes']=utils.getAttrsIfExists(_el2.div(class_name="_1g06"),['text'])
        el['comments']=utils.getAttrsIfExists(_el2.element(class_name='_1j-c'),['text'])
        el['url']=re.split(r"[\&\?]refid=",
                           utils.getAttrsIfExists(_el2.link(data_sigil="feed-ufi-trigger"),['href']))[0]
            # data_sigil="comments-token").text
        # for _el3 in _el2.element(data_gt='{"tn":"C"}').links(class_name="_4kk6"):
        #     el['usr']=_el3.text[:60]
        #     el['usrUrl']=re.split(r"[\&\?]refid=",_el3.href)[0]
        #     break
        #footer action bar second  line
    return el

# [x.text[:20] for x in b.articles()]
import pandas as pd
pd.DataFrame([getPostInfo(x) for x in b.articles()]).style.format(na_rep='-')

,pos,top_level_post_id,content_owner_id_new,key,usr,usrUrl,line2,text,likes,comments,url,photo_id,is_sponsored,page_id
0,1,3535007726785912,100008301576404,-473795450346053381,Ronnie Frostig,https://m.facebook.com/ronnie.frostig,6 hrs,"Thank you very much for the beautiful messages and warm wishes and making my 23 years at Publx Supermarket extra special. And also making me feel special and loved. Kindly accept my deepest appreciation to thank you for being so thoughtful and sweet. Thank you for all your love and support, and I couldn't have done it without you. It seems like yesterday that I started this journey. Thank you for keeping me in your heart and being such a big part of this journey.",Sandy Gage and 61 others,9 comments,https://m.facebook.com/story.php?story_fbid=pfbid029ebvBHteFq9mL3vb6dLVG5qcub1amwqcTe161mdLdTWrq2qv7hs4J23jdh1uZwjGl&id=100008301576404&m_entstream_source=feed_mobile&anchor_reactions=true,-,-,-
1,2,6115868455095880,100000183588400,-6158138528076713575,Sandeep Raghunath Tilekar,https://m.facebook.com/Sandeep.R.Tilekar,15 hrs,Enjoying my fitness journey,Geo Thomas and 27 others,12 comments,https://m.facebook.com/story.php?story_fbid=pfbid027sZp3c58zPk1eH6thb7hNk5pg7KE77ohp2Tp1u1W3imDQaPQCxJYo4m1kivM5Cbl&id=100000183588400&m_entstream_source=feed_mobile&anchor_reactions=true,6115868381762554,-,-
2,3,1746697602342636,210192189326526,-3606512130794509183,-,-,-,Download Our Ultimate Guide To Site Speed.,591,113 comments,https://m.facebook.com/story.php?story_fbid=pfbid02aJvkmdew7dae5BV2kVLKgiWrboyiEQPGsqsVjbUouLT5AjFS62Vd4mACugaHkPkhl&id=210192189326526&m_entstream_source=feed_mobile&anchor_reactions=true,-,1.000000,210192189326526
3,4,6232583713435100,100000506009128,7684996420479323132,Bal Mane,https://m.facebook.com/bal.mane.3,3 hrs,✍ Good morning friends 😊 काही दिवसांपूर्वी या पोस्टला सुंदर हस्ताक्षर समुहावर वाचकांचा भरघोस प्रतिसाद मिळाला होता. आज तीच पोस्ट इथेही देत आहे..,Makarand Ghangrekar and 20 others,23 comments,https://m.facebook.com/story.php?story_fbid=pfbid0A8ZXeQ17MNDWBcXbapNensuwKW7nkGfmLrWASS3vYNRxBDKpj2gKtw1Ddy5QH7P9l&id=100000506009128&m_entstream_source=feed_mobile&anchor_reactions=true,6232583646768440,-,-
4,5,-,-,5824257085895124885,Manoj Kulkarni,https://m.facebook.com/coolkarni22,6 hrs,Years back when I had seen my inspiration Tushar Karpe running at #DurshetForestMarathon I had a dream of running at this event. Not in my wildest dream had imagined that I could do a full marathon at this… More,Pranoti Patil and 80 others,32 comments,https://m.facebook.com/story.php?story_fbid=pfbid0D5oXtBRj35WkeF4hKUpv9VTTUrhT9pZ36Tu4m5tzzTQEiwQ2afRBsNMeR9FFKfVpl&id=100000274503750&m_entstream_source=feed_mobile&anchor_reactions=true,5708088675876893,-,-
5,6,3112957762348495,1595730204071266,7329820490527956930,-,-,-,Pick the Right Hosting for You,10K,336 comments,https://m.facebook.com/story.php?story_fbid=pfbid02szC6YH78eSBqRzGfzJowvoddyf6pDaBA57QKHoZwkayL6BfX8AtjiJMWgympz5ZEl&id=1595730204071266&m_entstream_source=feed_mobile&anchor_reactions=true,-,1.000000,1595730204071266
6,7,-,-,5279524595814210818,Vishakha Dharmadhikari,https://m.facebook.com/vishakha.dharmadhikari.3,8 hrs,"Quick trip to Banff and Jasper National Parks . Amazing landscape and views. Glaciers , snow clad mountains , glacier fed lakes and bright turquoise color of water makes it beautiful. Impossible to capture this beauty in camera.",Sanjay Kulkarni and 75 others,23 comments,https://m.facebook.com/story.php?story_fbid=pfbid0tBmaaz1QVE37quqTdCu1SgGnwpFMohbG1GwbhWYE3rudfjpxF2YQ8V4znw7r6cAxl&id=788941911&m_entstream_source=feed_mobile&anchor_reactions=true,10159987462041912,-,-
7,8,613929426762952,100044377001497,4543986505193033104,Ryan Hall,https://m.facebook.com/ryanhall3,7 hrs,"Hope. It’s the fuel of dreams. Do everything you can to cultivate it, keep it burning, and grow it.",727,11 comments,https://m.facebook.com/story.php?story_fbid=pfbid0u5Y1UHPgNdyfnZf4LkRZGXPkJQJStCcVrTZy4pn2fTbqycJEM5RqipL2AsVqEVA5l&id=10004437700149

In [64]:
# fb.login("pcmcrunners_fb")

In [ ]:
for x in b.elements(data_sigil= 'touchable ufi-inline-like like-reaction-flyout'.split())[:1]:
# for x in b.elements(class_name="_15ko")[:1]:
    x.click()
    print(x.attributes)

2022-08-21 00:30:41:ERROR:Internal Python error in the inspect module.
Below is the traceback from this internal error.

2022-08-21 00:30:41:INFO:
Unfortunately, your original traceback can not be constructed.

2022-08-21 00:30:41:ERROR:Internal Python error in the inspect module.
Below is the traceback from this internal error.

2022-08-21 00:30:41:INFO:
Unfortunately, your original traceback can not be constructed.

2022-08-21 00:30:41:ERROR:Internal Python error in the inspect module.
Below is the traceback from this internal error.

2022-08-21 00:30:41:INFO:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "C:\Users\AvinashMane\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-56-ab64d325dc55>", line 1, in <module>
    for x in b.elements(data_sigil= 'touchable ufi-inline-like like-reaction-flyout'.split())[:1]:
  File "c:\sw\python37\lib\site-packages\nerodia\container.py", line 13, in elements
    return HTMLElementCollection(self, self._extract_selector(*args, **kwargs))
TypeError: expected one of [<class 'str'>, <class 'bytes'>, <class 're.Pattern'>, <class 'bool'>], got ['touchable', 'ufi-inline-like', 'like-reaction-flyout']:<class 'list'>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\AvinashMane\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_

In [54]:
"logout"
from selenium.webdriver.common.keys import Keys
for _el in b.elements(text='Main Menu'):
    # _els.send_keys(os.environ["FB_2FA_CODE"])
    # _els.send_keys(Keys.ENTER)
    _el.execute_script("arguments[0].click();", _el)
    for _logoutButton in b.links(text='Log Out'):
        logging.info("Logging out")
        _logoutButton.execute_script("arguments[0].click();", _logoutButton)
        for _el3 in b.divs(text="Which account did you log out of?"):
            for _el4 in b.links(text="My Account"):
                _el4.click()
    # print(_els.attributes)
    

2022-08-21 00:29:15:INFO:Logging out


In [ ]:
li.scrolldown(9)

In [ ]:
importlib.reload(socialmediasite )
import socialmediasite

li=Linkedin(b,authFile=f'{AUTH}/auth.yaml')
utils.setLogger(f"{LOG}/strava-automation.log")

import pandas as pd
# .div(class_name='scaffold-finite-scroll').
nerodia.default_timeout=.1

def getPostData(postTag):
  "Return data regarding one post in linkedin feed"
  post={}
  if not postTag.exists:
    return
  _map={
      'post':'postTag.parent().element(class_name="visually-hidden").text',
      'urn':'postTag.attributes["data_urn"]',
      'name':'postTag.element(class_name="feed-shared-actor__name").text',
      'userUrl':'postTag.link(class_name="feed-shared-actor__container-link").href',
      'likes':'postTag.element(class_name="social-details-social-counts__reactions-count").text',
        # len('.div(class_name='feed-shared-social-actions').buttons(text='Like'))',
          # 'react-button--active' in 'react-button__text--like' in 
      'liked':'"react-button--active" in postTag.div(class_name="feed-shared-social-actions").button(text="Like").class_name',
      # print(name','.element(class_name='feed-shared-social-actions'
      #                                                            ).button(text='Like').class_name)
      'text':'postTag.element(class_name="feed-shared-update-v2__commentary").text',
      'desc':'postTag.element(class_name="feed-shared-actor__description").text',
      'subdesc':'postTag.element(class_name="feed-shared-actor__sub-description").text',
      # if post['subdesc'] not in ['Promoted']:
      'deg':'postTag.span(class_name="feed-shared-actor__supplementary-actor-info").text',
  }

  for _v,_e in _map.items():
    try:
        _m = re.search(r'.*(postTag\..*\))',_e)
        if _m:
            if eval(_m.group(1)+".exists"):
                post[_v]=eval(_e,{"postTag":postTag})
    except Exception as e:
        pass
        print(f"error {e!r} {_v} {_e}")
    
    if 'text' in post : post['text']=post['text'][:40]
    if 'userUrl' in post : post['userUrl']=post['userUrl'].split("?")[0]
    # post['liked']='react-button--active' in postTag.div(class_name='feed-shared-social-actions'
    #                                                          ).button(text='Like').class_name

  return post    

def getPostIds():
  "Get list of all feed posts"
  nerodia.default_timeout=.1
  posts=[]

  for x in b.elements(class_name='feed-shared-update-v2',
                      ):#data_id=re.compile("urn:li:activity:")):
    if re.match(r"^urn:li:activity:",x.data_urn):
        try:
            post=getPostData(x)
            post.update({'el':x})
            posts.append( post)
        except Exception as e:
          # Logs the error appropriately. 
            print(f">>{e!r}")

  return posts

posts = getPostIds() #li.


# posts=[]

# i=0
# for x in b.elements(class_name='feed-shared-update-v2',
#                     data_urn=re.compile("urn:li:activity:"),
#                    ):#data_id=re.compile("urn:li:activity:")):
#     # print (">>>",x.link(class_name="feed-shared-actor__container-link").href)
#     try:
#         print(x.h2(class_name="visually-hidden").text)
#         post=li.getPostData(x)
#         posts.append(post)
#     except Exception as e:
#         print(f"Error: {e!r}")
    # if i>3: 
    #     break
    # else:
    #     i+=1
# display (pd.DataFrame(posts))

In [ ]:
"list posts"
display (pd.DataFrame(posts)[['name','deg','text','liked']].style)

In [ ]:
for p in posts:
    if ('deg' in p and 
        (p['deg']=='• 1st')):
        p['text']=p['text'] if 'text' in p else ''
        if(not p['liked']):
            _likeButton=p['el'].div(class_name='feed-shared-social-actions'
                                  ).button(text='Like')
            if _likeButton.exists:  
                logging.info(f"Liking {p['name']} {p['text'][:30]}")
                try:
                    # _likeButton.click()
                    _likeButton.execute_script("arguments[0].click();", _likeButton)
                except Exception as e:
                  # Logs the error appropriately. 
                    print(f">>{e!r}")
        else:
            logging.warning(f"already liked {p['name']} {p['text'][:30]}")
    else:
        logging.warning(f"not 1st connection {p['name']}")


In [ ]:
"logout"
# li.logout()

# test linkedin

In [ ]:

Image(b.screenshot.png())

## A. Give likes in feed